# Scaling Data Analysis
## Using Dask with CSV data on single machine

In [1]:
import dask
import dask.dataframe as dd

In [2]:
dask.__version__

'0.10.0'

In [3]:
df = dd.read_csv('data/yellow_tripdata_2015-01.csv')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1.0,1.59,-73.993896,40.750111,1.0,N,-73.974785,40.750618,1.0,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1.0,3.30,-74.001648,40.724243,1.0,N,-73.994415,40.759109,1.0,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1.0,1.80,-73.963341,40.802788,1.0,N,-73.951820,40.824413,2.0,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1.0,0.50,-74.009087,40.713818,1.0,N,-74.004326,40.719986,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1.0,3.00,-73.971176,40.762428,1.0,N,-74.004181,40.742653,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [5]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          float64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID               float64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64


### What are the columns in this dataset?

In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

### What is the total number of passengers?

In [7]:
df.passenger_count.sum().compute()

21437303.0

### What are the different payment types and how many are there for each type?

In [8]:
df.payment_type.value_counts().compute()

1.0    7881388
2.0    4816992
3.0      38632
4.0      11972
5.0          2
Name: payment_type, dtype: int64

### Is there lack of a tip correleated with payment type 2?

In [9]:
df2 = df.assign(payment_2=(df.payment_type == 2),
                no_tip=(df.tip_amount == 0))[['no_tip', 'payment_2']]
df2.head()

,no_tip,payment_2
0,False,False
1,False,False
2,True,True
3,True,True
4,True,True


### What is the ratio of tip to fare?

In [10]:
df2 = df[(df.payment_type != 2) & (df.fare_amount > 0)]
df2 = df2.assign(tip_fraction=df2.tip_amount / df2.fare_amount)  # ratio of tip to fare

### What is the average % tip by hour of day?

In [12]:
df2.tpep_pickup_datetime = df2.tpep_pickup_datetime.astype('datetime64[ns]')

In [13]:
hour = df2.groupby(df2.tpep_pickup_datetime.dt.hour).tip_fraction.mean().compute()

### Plot results with Bokeh

In [17]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

fig = figure(title='Tip Fraction', 
             x_axis_label='Hour of day', 
             y_axis_label='Tip Fraction')
fig.line(x=hour.index, y=hour, line_width=3)
fig.y_range.start = 0

show(fig)

Loading BokehJS ...